## Extraction des caractéristiques et création des métadonnées

Dans ce notebook (`features-extraction.ipynb`), nous allons effectuer deux étapes principales :

1. **Création des métadonnées** : pour chaque image du dataset, nous allons enregistrer son nom de fichier (sans l’extension), ainsi que l’espèce (nom du dossier dans lequel elle se trouve).
2. **Extraction des caractéristiques** : nous utiliserons un modèle pré-entraîné (ResNet50) pour extraire des vecteurs de caractéristiques à partir des images.

Les images seront traitées dans un ordre aléatoire, mais les informations sur leur espèce d’origine et leur identifiant local seront conservées. Ceci permettra, par la suite, de visualiser facilement les résultats (par exemple, les images appartenant à un même cluster).

Le résultat final sera un fichier `.csv` contenant :
- Les vecteurs de caractéristiques de chaque image (chaque dimension dans une colonne `feature1`, `feature2`, ... , `feature2048`),
- L'espèce correspondante,
- L'identifiant de l’image dans son dossier d’origine.


### Importation des bibliothèques

Importation des modules nécessaires pour le traitement d’images, l’extraction de caractéristiques, et la gestion des données.


In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.preprocessing import LabelEncoder
import random


#### Chargement du modèle

Chargement du modèle ResNet50 pré-entraîné (sans la couche de classification) pour extraire des vecteurs de caractéristiques.


In [3]:
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

#### Extraction des caractéristiques

On parcourt les images du dataset de manière aléatoire ( pour que les clusters ne soient pas ordonnés ), on extrait leurs caractéristiques avec ResNet50, puis on les enregistre avec leur espèce et identifiant dans leur dossier


In [29]:


parent_folder = './Bird_Speciees_Dataset'


image_info = []

# Parcours des images
for subdir in os.listdir(parent_folder):
    subdir_path = os.path.join(parent_folder, subdir)
    if os.path.isdir(subdir_path):
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_info.append({
                    "image_name": img_name,
                    "species": subdir        
                })

# Mélange aléatoire 
random.shuffle(image_info)


# Extraction 
feature_list = []
for info in tqdm(image_info):
    img_path = os.path.join(parent_folder, info["species"], info["image_name"])
    img_name_no_ext = os.path.splitext(info["image_name"])[0] 
    
    try:
        img = image.load_img(img_path, target_size=(224, 224))  
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)  
        
        feature_vector = model.predict(x, verbose=0)[0] 
        
        feature_dict = {"species": info["species"], "id_in_class": img_name_no_ext}
        
        for i, feature in enumerate(feature_vector):
            feature_dict[f"feature{i+1}"] = feature
        
        feature_list.append(feature_dict)

    except Exception as e:
        print(f"Error processing {img_path}: {e}")



# Convertion to dataframe
features_df = pd.DataFrame(feature_list)



#verification
print(features_df.head())


100%|██████████| 811/811 [01:40<00:00,  8.06it/s]


              species id_in_class  feature1  feature2  feature3  feature4  \
0     EMPEROR PENGUIN         016  0.839631  0.367378  0.016061  0.000000   
1  AMERICAN GOLDFINCH         085  0.143122  0.187990  0.009073  0.216824   
2            FLAMINGO         079  0.166294  0.227275  0.024209  0.410724   
3  AMERICAN GOLDFINCH         117  0.787718  0.045966  0.995963  0.127777   
4     EMPEROR PENGUIN         127  0.346688  0.002053  0.000000  0.137830   

   feature5  feature6  feature7  feature8  ...  feature2039  feature2040  \
0  0.970196  0.030308  0.300561  0.073503  ...     0.033060     0.162204   
1  0.017523  1.073512  0.376696  1.023434  ...     0.062616     0.211144   
2  0.178875  0.117360  0.001518  0.186852  ...     0.842039     0.235661   
3  0.198726  1.341336  0.013681  0.647470  ...     0.785082     0.201939   
4  0.975613  0.008014  0.746616  0.254353  ...     0.000000     0.013387   

   feature2041  feature2042  feature2043  feature2044  feature2045  \
0     0.23

#### Division des données en ensembles d'entraînement et de test

Dans cette étape, les données ont été divisées en deux ensembles distincts : un ensemble d'entraînement (train) et un ensemble de test (test). Cette séparation est cruciale pour évaluer la performance du modèle de manière fiable, en s'assurant que l'ensemble de test n'est pas utilisé pendant l'entraînement.

##### Étapes réalisées :
1. **Division des données :** L'ensemble de données a été séparé en 70 % pour l'entraînement et 30 % pour le test, à l'aide de la fonction `train_test_split` de Scikit-learn.
2. **Sauvegarde des ensembles :** Les ensembles d'entraînement et de test ont été enregistrés dans deux fichiers CSV distincts : `bird_features_train.csv` et `bird_features_test.csv`.

**Résultat** :
Les ensembles d'entraînement et de test ont été créés et sauvegardés avec succès, prêts à être utilisés pour l'entraînement et l'évaluation de modèles de machine learning.


In [31]:
from sklearn.model_selection import train_test_split

# Division en test et train
train_df, test_df = train_test_split(features_df, test_size=0.3, random_state=42)

# Sauvegarde des deux
train_df.to_csv("bird_features_train.csv", index=False)
test_df.to_csv("bird_features_test.csv", index=False)

print("Les ensembles d'entraînement et de test ont été enregistrés avec succès.")


Les ensembles d'entraînement et de test ont été enregistrés avec succès.
